In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
from polire.random.random import Random
from polire.trend.trend import Trend
from polire.spline.bspline import BSpline
from polire.gp.gp import GaussianProcess
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib import colors
import seaborn as sns
import sys

In [44]:
df = pd.read_csv("../testdata/daily_44201_2018.csv")

In [45]:
df.head(3)

,State Code,County Code,Site Num,Parameter Code,POC,Latitude,Longitude,Datum,Parameter Name,Sample Duration,...,AQI,Method Code,Method Name,Local Site Name,Address,State Name,County Name,City Name,CBSA Name,Date of Last Change
0,1,3,10,44201,1,30.497478,-87.880258,NAD83,Ozone,8-HR RUN AVG BEGIN HOUR,...,21,NaN,-,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2019-02-12
1,1,3,10,44201,1,30.497478,-87.880258,NAD83,Ozone,8-HR RUN AVG BEGIN HOUR,...,30,NaN,-,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2019-02-12
2,1,3,10,44201,1,30.497478,-87.880258,NAD83,Ozone,8-HR RUN AVG BEGIN HOUR,...,41,NaN,-,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2019-02-12


In [46]:
some_value = 'Ozone'
df = df.loc[df['Parameter Name'] == some_value]

some_value = '2018-05-07'
df = df.loc[df['Date Local'] == some_value]

df = df.sample(frac=1)
df.shape

(1240, 29)

In [47]:
df_train, df_test = train_test_split(df, test_size=0.2)
df_train.head()

,State Code,County Code,Site Num,Parameter Code,POC,Latitude,Longitude,Datum,Parameter Name,Sample Duration,...,AQI,Method Code,Method Name,Local Site Name,Address,State Name,County Name,City Name,CBSA Name,Date of Last Change
79762,6,95,4,44201,1,38.102507,-122.237976,WGS84,Ozone,8-HR RUN AVG BEGIN HOUR,...,38,NaN,-,Vallejo,304 TUOLUMNE ST.,California,Solano,Vallejo,"Vallejo-Fairfield, CA",2019-01-28
54773,6,65,1010,44201,1,33.590833,-116.088333,WGS84,Ozone,8-HR RUN AVG BEGIN HOUR,...,84,NaN,-,Cabazon Resource Recovery Park Station,90-333 Avenue 63,California,Riverside,Not in a city,"Riverside-San Bernardino-Ontario, CA",2019-02-13
191447,24,17,10,44201,1,38.508547,-76.811864,WGS84,Ozone,8-HR RUN AVG BEGIN HOUR,...,54,NaN,-,Southern Maryland,14320 Oaks Road,Maryland,Charles,Not in a city,"Washington-Arlington-Alexandria, DC-VA-MD-WV",2019-01-09
107439,12,5,6,44201,1,30.130433,-85.731517,WGS84,Ozone,8-HR RUN AVG BEGIN HOUR,...,97,NaN,-,"ST.ANDREWS STATE PARK, PANAMA CITY BEACH","4607 STATE PARK LN PCB,FL",Florida,Bay,Lower Grand Lagoon,"Panama City, FL",2019-02-27
16659,4,13,9997,44201,1,33.503833,-112.095767,WGS84,Ozone,8-HR RUN AVG BEGIN HOUR,...,71,NaN,-,JLG SUPERSITE,4530 N 17TH AVENUE,Arizona,Maricopa,Phoenix,"Phoenix-Mesa-Scottsdale, AZ",2019-01-10


In [48]:
df.columns

Index(['State Code', 'County Code', 'Site Num', 'Parameter Code', 'POC',
       'Latitude', 'Longitude', 'Datum', 'Parameter Name', 'Sample Duration',
       'Pollutant Standard', 'Date Local', 'Units of Measure', 'Event Type',
       'Observation Count', 'Observation Percent', 'Arithmetic Mean',
       '1st Max Value', '1st Max Hour', 'AQI', 'Method Code', 'Method Name',
       'Local Site Name', 'Address', 'State Name', 'County Name', 'City Name',
       'CBSA Name', 'Date of Last Change'],
      dtype='object')

In [49]:
r = Random(resolution='standard')
trainX = df_train[['Longitude', 'Latitude']].values
trainy = df_train['Arithmetic Mean'].values
r.fit(trainX, trainy)

In [50]:
testX = df_test[['Longitude', 'Latitude']].values
testy = df_test['Arithmetic Mean'].values

In [51]:
x1max, x2max = np.max(trainX, axis=0)
print (x1max, x2max)
x1min, x2min = np.min(trainX, axis=0)

-66.126531 64.84568999999999


In [52]:
x1min, x1max, x2min, x2max

(-158.08861299999998, -66.126531, 18.440774, 64.84568999999999)

In [53]:
lss = [Random(), BSpline(kx=2, ky=2), Trend(order=2), GaussianProcess(), idw()]

NameError: name 'idw' is not defined

In [ ]:
for r in lss:
    r.fit(trainX, trainy)
#     try:
    Z = r.predict_grid((x1min, x1max), (x2min, x2max)).reshape(100, 100)
    sns.heatmap(Z)
    plt.title(r.__class__)
    plt.show()
    plt.close()
#     except:
#         print (f"Was not able to. for {r.__class__}!")


In [ ]:
plt.figure(figsize=(14, 8))
h = plt.scatter(trainX[:, 0], trainX[:, 1], c = trainy)
plt.title("The training set used.")
plt.colorbar(h)

In [ ]:
for r in lss:
    r.fit(trainX, trainy)
    try:
        fig, ax = plt.subplots(ncols=2, sharex=True, figsize=(16, 5))
        pred_y = r.predict(testX)
        vmin = min(testy)
        vmax = max(testy)
        norm = colors.Normalize(vmin=vmin, vmax=vmax)
        inter = ax[0].scatter(testX[:, 0], testX[:, 1], c = pred_y)
        ax[0].set_title(f"Interpolation {r.__class__}")
        gt = ax[1].scatter(testX[:, 0], testX[:, 1], c = testy)
        
        # normalize colorbar
        inter.set_norm(norm)
        gt.set_norm(norm)
        fig.colorbar(gt, ax=ax, orientation='horizontal', fraction=.1)
        
        ax[1].set_title("GT")
        plt.show()
        
    except NotImplementedError as e:
        print (f"Was not able to. for {r.__class__}!")
        print(str(e))

In [ ]:
type(testX[:, 0])

In [ ]:
r.predict

In [ ]:
lims = 1, 3, 3, 5
resolution = 12
x1min, x1max, x2min, x2max = lims
x1 = np.linspace(x1min, x1max, resolution)
x2 = np.linspace(x2min, x2max, resolution)
X1, X2 = np.meshgrid(x1, x2)